In [18]:
from ase import Atoms
from ase.build import fcc111
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms, FixConstraint
from ase.io import write
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
import numpy as np

class LockXYManualZ(FixConstraint):
    """
    Constraint that:
      1) Fixes x,y for selected atoms (overrides them to old positions).
      2) Zeros out forces and momenta for these atoms so MD doesn't move them
         in any direction (including z).
      3) Leaves z-coordinate unchanged by MD updates, so a manual displacement
         (apply_strain) is the only way to change z.
    """
    def __init__(self, indices):
        super().__init__()
        self.indices = np.asarray(indices, dtype=int)

    def get_indices(self):
        return self.indices

    def get_removed_dof(self, atoms):
        """
        We remove 3 degrees of freedom per atom from ASE's viewpoint,
        so integrators won't update them. (Alternatively, you could
        remove fewer DOFs if you *did* want them to move in z via MD.)
        """
        return 3 * len(self.indices)

    def adjust_positions(self, atoms, new_positions):
        """Lock x,y to old values.  Do NOT override z."""
        old_positions = atoms.positions
        for i in self.indices:
            new_positions[i, 0] = old_positions[i, 0]
            new_positions[i, 1] = old_positions[i, 1]
        # z is untouched, so any manual displacement remains in effect.

    def adjust_forces(self, atoms, forces):
        """
        Zero out forces on these atoms.  If you want them
        to remain immobile under MD, including no z motion,
        then you must remove the net force in all directions.
        """
        forces[self.indices] = 0.0

    def adjust_momenta(self, atoms, momenta):
        """
        Zero out velocities on these atoms so they don't
        drift in z or pick up velocity from forces. 
        Otherwise, a leftover velocity in z could cause motion.
        """
        momenta[self.indices] = 0.0


# ----------------------------------------------------------
# Create an FCC aluminum slab
# ----------------------------------------------------------
size = (4, 4, 20)
slab = fcc111('Al', size=size, vacuum=10.0, periodic=False)
del slab[42]  # remove a random atom for a 'weakness'

import random

def randomize_atoms_to_cu(slab, num_cu_atoms, seed=None):
    """
    Randomly replace some atoms in the slab with Cu.

    Parameters:
    - slab (Atoms): The Atoms object to modify.
    - num_cu_atoms (int): The number of atoms to replace with Cu.
    - seed (int, optional): Seed for the random number generator to ensure reproducibility.
    """
    if seed is not None:
        random.seed(seed)  # Set the seed for reproducibility

    total_atoms = len(slab)
    if num_cu_atoms > total_atoms:
        raise ValueError("num_cu_atoms exceeds the total number of atoms in the slab.")

    # Select random unique indices
    indices_to_replace = random.sample(range(total_atoms), num_cu_atoms)
    for idx in indices_to_replace:
        slab[idx].symbol = 'Cu'
    print(f"Replaced {num_cu_atoms} atoms with Cu at indices: {indices_to_replace}")

# ----------------------------------------------------------
# Replace some atoms with Cu (with reproducibility)
# ----------------------------------------------------------
num_cu_atoms = int(0.5*len(slab))  # Specify the number of Cu atoms to introduce
random_seed = 42  # Specify a seed for reproducibility
randomize_atoms_to_cu(slab, num_cu_atoms, seed=random_seed)


slab.set_calculator(EMT())

# Identify bottom and top layers
positions = slab.get_positions()
z_positions = positions[:, 2]
bottom_threshold = np.min(z_positions) + 5.0
bottom_mask = z_positions < bottom_threshold
top_threshold = np.max(z_positions) - 5.0
top_indices = np.where(z_positions > top_threshold)[0]

# Freeze bottom few layers
bottom_constraint = FixAtoms(mask=bottom_mask)

# Lock XY for top atoms, while letting us manually move z
top_constraint = LockXYManualZ(top_indices)

# Add constraints to slab
slab.set_constraint([bottom_constraint, top_constraint])

# ----------------------------------------------------------
# MD & Strain parameters
# ----------------------------------------------------------
displacement_rate = 0.01
n_steps = 5000
strain_interval = 10
save_interval = 100
output_file = "fcc_al_tensile_test.xyz"

# Initialize velocities at 300K
MaxwellBoltzmannDistribution(slab, temperature_K=300)

# Set up Langevin MD
dyn = Langevin(slab, 0.1, temperature_K=300, friction=0.01)

# Progress
current_step = 0

def apply_strain():
    """
    Manually shift top atoms in z. Because the top_constraint
    zeroes out all velocities & forces, MD won't otherwise 
    alter these z positions.
    """
    pos = slab.get_positions()
    pos[top_indices, 2] += displacement_rate
    slab.set_positions(pos)

def save_trajectory():
    global current_step
    current_step += save_interval
    write(output_file, slab, format='xyz', append=True)
    print(f"Saving: Step {current_step}/{n_steps}")

dyn.attach(apply_strain, interval=strain_interval)
dyn.attach(save_trajectory, interval=save_interval)

with open(output_file, 'w') as f:
    f.write("")

print("Starting tensile test simulation...")
dyn.run(n_steps)
print(f"Simulation complete. Trajectory saved to", output_file)


Replaced 159 atoms with Cu at indices: [57, 12, 140, 125, 114, 71, 52, 279, 44, 302, 216, 16, 15, 47, 111, 119, 258, 13, 287, 101, 311, 214, 112, 229, 142, 3, 81, 308, 174, 294, 79, 110, 172, 312, 305, 194, 49, 183, 176, 135, 22, 235, 274, 63, 193, 40, 150, 185, 98, 35, 23, 116, 148, 273, 303, 51, 283, 289, 232, 186, 83, 189, 181, 107, 171, 68, 179, 239, 290, 165, 18, 155, 162, 43, 136, 259, 62, 41, 118, 97, 69, 236, 163, 280, 261, 56, 175, 309, 215, 196, 198, 14, 58, 210, 8, 206, 80, 102, 253, 307, 54, 145, 281, 222, 218, 167, 127, 299, 164, 117, 36, 67, 269, 275, 190, 143, 137, 207, 191, 149, 109, 199, 221, 92, 233, 223, 130, 126, 268, 317, 28, 39, 160, 265, 250, 108, 152, 219, 270, 251, 182, 264, 298, 64, 141, 2, 29, 202, 220, 225, 87, 188, 75, 304, 271, 267, 0, 201, 128]
Starting tensile test simulation...
Saving: Step 100/5000


/tmp/ipykernel_2128156/937542056.py:98: DeprecationWarning: Please use atoms.calc = calc
  slab.set_calculator(EMT())


Saving: Step 200/5000
Saving: Step 300/5000
Saving: Step 400/5000
Saving: Step 500/5000
Saving: Step 600/5000
Saving: Step 700/5000
Saving: Step 800/5000
Saving: Step 900/5000
Saving: Step 1000/5000
Saving: Step 1100/5000
Saving: Step 1200/5000
Saving: Step 1300/5000
Saving: Step 1400/5000
Saving: Step 1500/5000
Saving: Step 1600/5000
Saving: Step 1700/5000
Saving: Step 1800/5000
Saving: Step 1900/5000
Saving: Step 2000/5000
Saving: Step 2100/5000
Saving: Step 2200/5000
Saving: Step 2300/5000
Saving: Step 2400/5000
Saving: Step 2500/5000
Saving: Step 2600/5000
Saving: Step 2700/5000
Saving: Step 2800/5000
Saving: Step 2900/5000
Saving: Step 3000/5000
Saving: Step 3100/5000
Saving: Step 3200/5000
Saving: Step 3300/5000
Saving: Step 3400/5000
Saving: Step 3500/5000
Saving: Step 3600/5000
Saving: Step 3700/5000
Saving: Step 3800/5000
Saving: Step 3900/5000
Saving: Step 4000/5000
Saving: Step 4100/5000
Saving: Step 4200/5000
Saving: Step 4300/5000
Saving: Step 4400/5000
Saving: Step 4500/5